In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import re
import sys

import os
print(os.getcwd())   # Prints the current working directory

C:\Users\user\Desktop\Twitter NLP Classification Model\Bro Isep


In [2]:
# Provide the new path here
os.chdir('C:\\Users\\user\\Desktop\\Twitter NLP Classification Model\\DATASET\\dataset 31-05-2020')

df = pd.read_csv('dataset.csv')
df_normalization = pd.read_csv('normalization.csv')
dict_1 = df_normalization.set_index('kata tidak baku').T.to_dict('list')
df_stopword = pd.read_csv('stopword.csv')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  if __name__ == '__main__':


In [44]:
def normalize_fcn(data):
    for key, value in dict_1.items():
        data = data.replace(' '+key+' ',' '+value[0]+' ')
    return data

In [18]:
df = df.assign(Text2 = df['Text'])                                               # Add new comparison column text

df['Text2'] = df['Text'].replace(r'\\n',' ', regex=True)                         # remove new line 
df['Text2'] = df['Text2'].map(lambda x: re.sub(r'http\S+', ' ', x))              # remove url
df['Text2'] = df['Text2'].map(lambda x: re.sub(r'#\S+', ' ', x))                 # remove hashtag
df['Text2'] = df['Text2'].map(lambda x: re.sub(r'@\S+', ' ', x))                 # remove username
df['Text2'] = df['Text2'].map(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x)) # remove any digit
df['Text2'] = df['Text2'].map(lambda x: re.sub(r'\\\S+', ' ', x))                # remove \u00a0 and the other similar code
df['Text2'] = df['Text2'].map(lambda x: re.sub(r'[^A-Za-z0-9]+', ' ', x))        # remove special character including emoji
df['Text2'] = df['Text2'].str.replace('RT','',case=True)                         # remove RT
df['Text2'] = df['Text2'].str.lstrip()                                           # remove whitespace,newline,tab,basically everything from the start of string
df['Text2'] = df['Text2'].str.rstrip()                                           # remove whitespace,newline,tab,basically everything from the end of string
df['Text2'] = df['Text2'].str.lower()                                            # lowercase


df = df.assign(Text3 = df['Text2'])                                              # Create new text for display normalized result
df['Text3'] = df['Text2'].map(lambda x: normalize_fcn(x))                        # Normalize

######################################
#Remove Number
######################################
df = df.assign(RemoveNumber = df['Text3'])
df['RemoveNumber'] = df['Text3'].map(lambda x: re.sub('[0-9]+', ' ', x) )         # remove Number

######################################
# Stopword
#######################################
#df = df.assign(Text4 = df['Text3'])                                              # Create new text for display stop word removal result
df = df.assign(Text4 = df['RemoveNumber'])                                        # Create new text for display stop word removal result
stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopword = df_stopword['Stopword'].to_numpy()
more_stopword = list(more_stopword) 
# Merge stopword
data_stopword = stop_factory + more_stopword
dictionary_stopword = ArrayDictionary(data_stopword)
str1 = StopWordRemover(dictionary_stopword)
#df['Text4'] = df['Text3'].map(lambda x: str1.remove(x))                         # Stop word
df['Text4'] = df['Text4'].map(lambda x: str1.remove(x))                         # Stop word
df['Text4'] = df['Text4'].str.lstrip()                                          # remove whitespace,newline,tab,basically everything from the start of string
df['Text4'] = df['Text4'].str.rstrip()                                          # remove whitespace,newline,tab,basically everything from the end of string

#####################################
#Stemming
#####################################
df = df.assign(Text5 = df['Text4'])  
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process
df['Text5'] = df['Text5'].map(lambda x:stemmer.stem(x))
df['Text5'] = df['Text5'].str.lstrip()                                          # remove whitespace,newline,tab,basically everything from the start of string
df['Text5'] = df['Text5'].str.rstrip()                                          # remove whitespace,newline,tab,basically everything from the end of string

# print(df.head(25))
df.to_csv('result_prepro3.csv')